In [ ]:
import subprocess
import pandas as pd
import time

# ===========================
# Load Dataset
# ===========================
df = pd.read_csv("moviekaggle.csv")

# Ensure only first 30 rows
df = df.head(30).copy()

# Add new columns
df["gemma_output"] = ""
df["correct"] = 0

# ===========================
# Function to query Gemma 3 (4B)
# ===========================
def ask_gemma(review_text):
    prompt = f"""
Classify the sentiment of this review as Positive or Negative:
{review_text}
Answer with only one word.
"""

    result = subprocess.run(
        ["ollama", "run", "gemma3:4b"],    # <-- MODEL CHANGED HERE
        input=prompt.encode("utf-8"),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

    output = result.stdout.decode("utf-8").strip().lower()
    return output


# ===========================
# Run Sentiment Test
# ===========================
correct_count = 0

for i in range(len(df)):
    review = df.loc[i, "review"]
    true_sentiment = df.loc[i, "sentiment"].strip().lower()

    print(f"\n==============================")
    print(f"Processing Review #{i+1}")
    print(f"Text: {review[:250]}...")
    print(f"True Label: {true_sentiment}")

    gemma_ans = ask_gemma(review)

    print(f"Gemma Output: {gemma_ans}")

    # Normalize model answer
    if "positive" in gemma_ans:
        final_pred = "positive"
    elif "negative" in gemma_ans:
        final_pred = "negative"
    else:
        final_pred = "unknown"

    df.loc[i, "gemma_output"] = final_pred

    # Compare with true value
    if final_pred == true_sentiment:
        df.loc[i, "correct"] = 1
        correct_count += 1
        print("Match Status: ✔ CORRECT")
    else:
        df.loc[i, "correct"] = 0
        print("Match Status: ✘ WRONG")

    time.sleep(0.5)   # for clean printing


# ===========================
# Calculate Accuracy
# ===========================
accuracy = correct_count / len(df)

print("\n=================================")
print("           FINAL RESULTS")
print("=================================")
print(f"Total Samples: {len(df)}")
print(f"Correct Predictions: {correct_count}")
print(f"Accuracy: {accuracy:.4f}")
print("=================================\n")

# ===========================
# Save the output file
# ===========================
df.to_csv("moviekaggle_gemma_results.csv", index=False)

print("Saved results to moviekaggle_gemma_results.csv")

